<a href="https://colab.research.google.com/github/tanujsaini05/Solution-Challenge/blob/main/Fauna_soln_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
from helper_functions import confusion_matrix,plot_loss_curves,create_tensorboard_callback

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
train_dir='/content/drive/MyDrive/Animalin/Train'
test_dir='/content/drive/MyDrive/Animalin/Test'

In [ ]:
datagen=ImageDataGenerator(rescale = 1./255)

In [ ]:
train_data=datagen.flow_from_directory(train_dir,target_size = (224, 224),
    shuffle = True,
    batch_size=32)

test_data=datagen.flow_from_directory(test_dir,target_size=(224,224),shuffle=True,batch_size=32)

Found 5891 images belonging to 9 classes.
Found 461 images belonging to 9 classes.


In [ ]:
print(test_data.class_indices)

{'American Bully': 0, 'Buffalo': 1, 'Cat': 2, 'Cow': 3, 'German Shepherd': 4, 'Golden Retriever': 5, 'Labrador': 6, 'Pug': 7, 'Rottweiler': 8}


In [ ]:
Early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1,patience=3)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint('model_checkpoints/final_one/cp_3.ckpt',save_best_only=True,monitor='val_loss',)

Reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=0.2,min_lr=1e-7,verbose=1)

In [ ]:
base_resnet= tf.keras.applications.ResNet50V2(include_top=False)

94668760/94668760 [==============================] - 0s 0us/step


In [ ]:
i=0
for layer in base_resnet.layers:
  #print(f"layer_name = {layer.name}  and number is {i}")
  if i>51 or i==0:
    layer.trainable=False
  i+=1
for layer in base_resnet.layers:
  print(layer.trainable)



False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
F

In [ ]:
model_resnet = tf.keras.Sequential()
model_resnet.add(base_resnet)
model_resnet.add(tf.keras.layers.GlobalAveragePooling2D())
model_resnet.add(tf.keras.layers.Dense(9, activation='softmax'))

In [ ]:
model_resnet.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
model_resnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, None, None, 2048   23564800  
                             )                                   
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 9)                 18441     
                                                                 
Total params: 23583241 (89.96 MB)
Trainable params: 622729 (2.38 MB)
Non-trainable params: 22960512 (87.59 MB)
_________________________________________________________________


In [ ]:
hist_model_resnet=model_resnet.fit(train_data,validation_data=test_data,epochs=50,batch_size=32,verbose=1,callbacks=[Early_stopping,Reduce_lr,model_checkpoint])

Epoch 1/50
185/185 [==============================] - 1734s 9s/step - loss: 1.1904 - accuracy: 0.5867 - val_loss: 0.7944 - val_accuracy: 0.7137 - lr: 0.0010
Epoch 2/50
185/185 [==============================] - 530s 3s/step - loss: 0.4325 - accuracy: 0.8503 - val_loss: 0.6440 - val_accuracy: 0.7852 - lr: 0.0010
Epoch 3/50
185/185 [==============================] - 545s 3s/step - loss: 0.3082 - accuracy: 0.8929 - val_loss: 0.4765 - val_accuracy: 0.8395 - lr: 0.0010
Epoch 4/50
185/185 [==============================] - 542s 3s/step - loss: 0.2095 - accuracy: 0.9255 - val_loss: 0.3794 - val_accuracy: 0.8698 - lr: 0.0010
Epoch 5/50
185/185 [==============================] - ETA: 0s - loss: 0.2518 - accuracy: 0.9150
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
185/185 [==============================] - 519s 3s/step - loss: 0.2518 - accuracy: 0.9150 - val_loss: 0.3918 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 6/50
185/185 [==============================] - 539

In [ ]:
from helper_functions import confusion_matrix

In [ ]:
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.io.decode_image(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img
load_and_prep_image(filename='1.jpg')

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.4065989 , 0.43902186, 0.28096864],
        [0.40436485, 0.46140268, 0.2290735 ],
        [0.37457982, 0.42948177, 0.22163865],
        ...,
        [0.88235295, 0.88235295, 0.8901961 ],
        [0.88235295, 0.88235295, 0.8901961 ],
        [0.88235295, 0.88235295, 0.8901961 ]],

       [[0.72935295, 0.7397521 , 0.70492506],
        [0.42139295, 0.47664502, 0.2645302 ],
        [0.3465336 , 0.40143555, 0.19359243],
        ...,
        [0.88235295, 0.88235295, 0.8901961 ],
        [0.88235295, 0.88235295, 0.8901961 ],
        [0.88692784, 0.88692784, 0.894771  ]],

       [[0.7382784 , 0.7455263 , 0.7337347 ],
        [0.41095936, 0.4662115 , 0.25009438],
        [0.3395952 , 0.3944972 , 0.18665403],
        ...,
        [0.884944  , 0.884944  , 0.89278716],
        [0.8862745 , 0.8862745 , 0.89411765],
        [0.8862745 , 0.8862745 , 0.89411765]],

       ...,

       [[0.6967621 , 0.61440915, 0.53989935],
        [0.64

In [ ]:

pred_probs = model_resnet.predict(test_data, verbose=1)

In [ ]:
pred_classes = pred_probs.argmax(axis=1)

In [ ]:
confusion_matrix(y_true=y_labels,y_pred=pred_classes)

In [ ]:
model_resnet.save("/content/drive/MyDrive/Animalin/Train/one.keras")

In [ ]:
!mkdir -p saved_model
model_resnet.save('/content/drive/MyDrive/Animalin/Train/one.keras')

In [ ]:
model_resnet.save('my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Animalin/Train/one.keras')

In [ ]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, None, None, 2048   23564800  
                             )                                   
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 9)                 18441     
                                                                 
Total params: 23583241 (89.96 MB)
Trainable params: 622729 (2.38 MB)
Non-trainable params: 22960512 (87.59 MB)
_________________________________________________________________


In [ ]:
new_model.evaluate(test_data)

15/15 [==============================] - 18s 1s/step - loss: 0.2463 - accuracy: 0.9024


[0.24625317752361298, 0.9023861289024353]

In [ ]:
prediction=new_model.predict(tf.expand_dims(load_and_prep_image("/content/drive/MyDrive/Animalin/Train/Labrador/aug_0_1004.jpg"),axis=0)).argmax()
a=["American Bully",'Buffalo',"Cat","Cow","German Shephard","Golden Retriever","Labrador","Pug","Rottweiler"]
print(f"prediction is {a[prediction]}")

1/1 [==============================] - 0s 23ms/step
prediction is Labrador
